## Анализ временных рядов, часть 1


В данной задаче вы будете работать с данными температуре воздуха. Таблица содержит данные погоды примерно за 5 лет для городов США, Канады и Израиля.
Файл `temperature.csv` содержит температуру в градусах Кельвина.




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

Загрузите таблицу. Обратиет внимание на параметр `parse_dates`, он используется для приведения столбца с датами к правильному типу данных.

In [ ]:
df = pd.read_csv('temperature.csv', parse_dates=['datetime'])

Установим дату как индекс

In [ ]:
df = df.set_index('datetime')
df.head()

Посмотрим, как выглядят наши данные.

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df['Portland'], lw=0.5);

Какие закономерности вы видите на графике?

**Ответ:**

Данные представляют собой циклически повторяющиеся значения. В начале каждого года температура достигает своего минимума, а к середине года достигает максимума. Температурные показатели остаются в пределах определенного диапазона и повторяются из года в год. Таким образом, для прогнозирования таких данных можно применять решающие деревья и случайный лес.





Посчитайте пропуски для каждого из городов.

In [ ]:
df.isna().sum()

Выберите город с наименьшим количеством пропусков. Назовем этот город X.

In [ ]:
X = df['Detroit']
X.head()

Если данные имеют пропуски в начале или в конце, то лучше обрезать данные. Остальные пропуски стоит заполнить предыдущим значением по дате. Для этого вам может помочь функция `pandas.DataFrame.fillna`.

In [ ]:
X = X.iloc[1:]
X.shape

Для города X будем предсказывать температуру с помощью моделей машинного обучения.


Разбейте данные на обучение и тест. Для тестовой выборки используйте последние **365 дней** из данных (не забывайте, что в данных представлены показатели за каждый час).


In [ ]:
train = X[:-365*24]
test = X[-365*24:]
assert train.shape[0] + test.shape[0] == X.shape[0]

С форматом даты модели работать не умеют, поэтому нам необходимо описать дату признаками. Сделаем преобразование даты в признаки.

In [ ]:
def create_date_features(date):
    """Создает фичи из даты"""
    row = {}
    row["dayofweek"] = date.dayofweek
    row["quarter"] = date.quarter
    row["month"] = date.month
    row["year"] = date.year
    row["dayofyear"] = date.dayofyear
    row["dayofmonth"] = date.day
    row["weekofyear"] = date.weekofyear
    return row

def create_only_date_train_features(y_series):
    """
    Создает обучающий датасет из признаков, полученных из дат для y_series
    """
    time_features = pd.DataFrame(
    [create_date_features(date) for date in y_series.index] )
    return time_features, y_series

In [ ]:
X_train, y_train = create_only_date_train_features(train)
X_test, y_test = create_only_date_train_features(test)

In [ ]:
X_train

Постройте и обучите три модели:
* модель линейной регрессии
* решающего дерева
* случайного леса.

In [ ]:
lr = LinearRegression()
tree = DecisionTreeRegressor()
forest = RandomForestRegressor()

models = [(lr, 'Linear Regression'), (tree, 'Decision Tree'), (forest, 'Random Forest')]

for model, i in models:
    model.fit(X_train, y_train)

Посчитайте качество на тесте (MSE, MAE, MAPE).



In [ ]:
y_preds = []

for model, name in models:
    y_pred = model.predict(X_test)
    y_preds.append(y_pred)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'{name}')
    print(f'MSE: {mse.round(3)}')
    print(f'MAE: {mae.round(3)}')
    print(f'MAPE: {mape.round(3)}')
    print()

Визуализируйте результат.

In [ ]:
def plot_results(y_to_train, y_to_test, y_forecast, model_name):
    """
    Функция для визуализации временного ряда и предсказания.
    Параметры:
        - y_to_train: pd.Series
            Временной ряд, на котором обучалась модель.
        - y_to_test: pd.Series
            Временной ряд, который предсказывает модель.
        - y_forecast: array
            Предсказания модели.
        - plot_conf_int: bool
            Надо ли строить предсказательного интервал.
        - left_bound: array
            Левая граница предсказательного интервала.
        - right_bound: array
            Правая граница предсказательного интервала.
    """
    plt.figure(figsize=(15, 5))
    plt.title(f"Показатели температуры для города Х \n {model_name}", fontsize=15)
    plt.plot(y_to_train, label="train")
    plt.plot(y_to_test, label="test")
    plt.plot(y_to_test.index, y_forecast, label="prediction")
    plt.legend()
    plt.show()

In [ ]:
plot_results(y_train, y_test, y_preds[0], models[0][1])

In [ ]:
plot_results(y_train, y_test, y_preds[1], models[1][1])

In [ ]:
plot_results(y_train, y_test, y_preds[2], models[2][1])

Сделайте выводы.

**Вывод:**

На метриках видно, что линейная регрессия показывает наихудшие результаты. Решающее дерево и случайный лес демонстрируют схожие показатели по метрикам и визуально на графике проявляют схожее поведение. Единственное различие заключается в том, что случайный лес выглядит немного более точным и лучше согласуется с реальными данными. В общем, можно сказать, что дерево и лес достаточно неплохо моделируют реальные данные.